In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygplates
import pygmt
import seaborn as sns
from pyrolite.geochem.norm import get_reference_composition

import sys
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy

%matplotlib inline
%load_ext autoreload
%autoreload 2

f = get_reference_composition('Chondrite_SM89')


In [ ]:
df_Tang = pd.read_excel('../datafiles/abf1876-tang-sm-database-s1.xlsx', sheet_name='Filtered')

df_Tang.Continent.unique()

df_Tang = df_Tang[(df_Tang.Continent.str.contains('S America')) & (df_Tang.Age<500)]


In [ ]:
dfLee41 = pd.read_excel('../datafiles/zircons//Lee/Appendix_Tables.edt.xls', sheet_name='Appendix 4', skiprows=1)
dfLee42 = pd.read_excel('../datafiles/zircons//Lee/Appendix_Tables.edt.xls', sheet_name='Appendix 4_2', skiprows=1)

dfLee41.columns.values

In [ ]:
dfN = pd.read_excel('../datafiles/zircons//Nathwani/410_2020_1766_MOESM2_ESM.xlsx', 
                    sheet_name='3. Zircon U-Pb and Traces', skiprows=2)

f = get_reference_composition('Chondrite_SM89')

euN = dfN['Eu151_ppm'] / f.comp['Eu'].value
smN = dfN['Sm147_ppm'] / f.comp['Sm'].value
gdN = dfN['Gd157_ppm'] / f.comp['Gd'].value

dfN['eu_eu_star'] = euN / np.sqrt(smN * gdN)



In [ ]:
dfJ = pd.read_excel('../datafiles/zircons//Jara/ScienceDirect_files_17Dec2021_06-35-22.334/1-s2.0-S1342937X2100023X-mmc2.xlsx', 
                    skiprows=3, skipfooter=6, header=[0, 1])

dfJ.replace({'BDL': np.nan}, inplace=True)

dfJ['PrefAge'] = dfJ['Pref. Age']['(Ma)']

euN = dfJ['Eu153']['Mean (ppm)'] / f.comp['Eu'].value
smN = dfJ['Sm147']['Mean (ppm)'] / f.comp['Sm'].value
gdN = dfJ['Gd157']['Mean (ppm)'] / f.comp['Gd'].value

dfJ['eu_eu_star'] = euN / np.sqrt(smN * gdN)
dfJ


In [ ]:
dfCM = pd.read_excel('../datafiles/zircons//ChelleMichou/1-s2.0-S0024493714001005-mmc2.xlsx', 
                         sheet_name='Trace elements')

dfCM.dropna(inplace=True)
dfCM['age_ma'] = np.nan

dfCM['age_ma'].iloc[dfCM['Lazer spot'].str.contains('CC055')] = 40.374
dfCM['age_ma'].iloc[dfCM['Lazer spot'].str.contains('CC038')] = 40.211
dfCM['age_ma'].iloc[dfCM['Lazer spot'].str.contains('CC022')] = 35.633
dfCM['age_ma'].iloc[dfCM['Lazer spot'].str.contains('CC051')] = 35.597
dfCM['age_ma'].iloc[dfCM['Lazer spot'].str.contains('CC056')] = 35.20

euN = dfCM['Eu (ppm)'] / f.comp['Eu'].value
smN = dfCM['Sm (ppm)'] / f.comp['Sm'].value
gdN = dfCM['Gd (ppm)'] / f.comp['Gd'].value

dfCM['eu_eu_star'] = euN / np.sqrt(smN * gdN)


In [ ]:
dfMunoz1 = pd.read_excel('../datafiles/zircons//Munoz/MunozZirconData.xlsx', sheet_name='Sheet1')
dfMunoz2 = pd.read_excel('../datafiles/zircons//Munoz/MunozZirconData.xlsx', sheet_name='Sheet2')

dfMunoz1['Sample'] = dfMunoz1.Sample.ffill()
dfMunoz2['Sample'] = dfMunoz2.Sample.ffill()

dfMunoz1.dropna(subset=['Eu'], inplace=True)
dfMunoz2.dropna(subset=['Spot'], inplace=True)

dfMunoz1.replace({'·': '.', '\xa0': ''}, regex=True, inplace=True)
dfMunoz2.replace({'·': '.', '\xa0': ''}, regex=True, inplace=True)


age_list = []
th_u_list = []
for i,row in dfMunoz1.iterrows():
    age = dfMunoz2[(dfMunoz2.Sample==row.Sample) & (dfMunoz2.Spot.astype(float)==row.Spot)]['Age']
    age_list.append(float(age.values[0]))
    th_u_list.append(float(dfMunoz2[(dfMunoz2.Sample==row.Sample) & (dfMunoz2.Spot.astype(float)==row.Spot)]['Th/U']))

dfMunoz1['age_ma'] = age_list


In [ ]:
plt.figure(figsize=(14,5))
plt.plot(df_Tang.Age, df_Tang['Eu/Eu*zr'], 'ko', label='Tang++2021')
plt.plot(dfLee41['Age (Ma) ±1s'], dfLee41['EuN /EuN*'], 'o', alpha=0.5, label='Lee++')
plt.plot(dfLee42['Age (Ma) ± 1s'], dfLee42['EuN /EuN*'], 'o', alpha=0.5, label='Lee++')
plt.plot(dfN['Weighted Mean Age (Ma)'], dfN['eu_eu_star'], 'o', alpha=0.7, label='Nathwani++')
plt.plot(dfJ['PrefAge'], dfJ['eu_eu_star'], 'o', alpha=0.3, label='Jara++')
plt.plot(dfCM['age_ma'], dfCM['eu_eu_star'], 'o', alpha=0.3, label='ChelleMichou++')
plt.plot(dfMunoz1['age_ma'], dfMunoz1['EuN/ EuN*'].astype(float), 'o', alpha=0.3, label='Munoz++')
#plt.xlim(30,70)
plt.xlim(0,400)
plt.ylim(0,0.9)
plt.legend()


In [ ]:
ages_cat = []
eu_cat = []
ages_list = [df_Tang.Age.tolist(), 
            dfLee41['Age (Ma) ±1s'].tolist(), 
            dfLee42['Age (Ma) ± 1s'].tolist(), 
            dfN['Weighted Mean Age (Ma)'].tolist(),
            dfJ['PrefAge'].tolist(),
            dfCM['age_ma'].tolist(),
            dfMunoz1['age_ma'].tolist()]

eu_list = [df_Tang['Eu/Eu*zr'],
           dfLee41['EuN /EuN*'],
           dfLee42['EuN /EuN*'],
           dfN['eu_eu_star'],
           dfJ['eu_eu_star'],
           dfCM['eu_eu_star'],
           dfMunoz1['EuN/ EuN*'].astype(float)]

for ages,eu in zip(ages_list,eu_list):
    ages_cat.extend(ages)
    eu_cat.extend(eu)
    
dff = pd.DataFrame(data={'age': ages_cat,
                         'eu_eu': eu_cat}).dropna()

#dff

dff['age_bin'] = np.round(dff['age']/10.)*10.
dff['thickness'] = 84.2 * dff['eu_eu'] + 24.5

dff = dff[dff['age_bin']<350.]

plt.figure(figsize=(16,6))
sns.boxenplot(x='age_bin', y='eu_eu', data=dff)
plt.show()
